In [1]:
import numpy as np
import pandas as pd
import re
import os
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import torch
import lxml.html
import nltk
import shutil
import orjsonl

from nltk.corpus import stopwords
nltk.download('stopwords')


from lxml_html_clean import Cleaner
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from transformers import BertForSequenceClassification, BertTokenizer, get_linear_schedule_with_warmup
from datasets import Dataset
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModelForMaskedLM, AutoTokenizer

#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1000"


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrea.Bagante\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Andrea.Bagante\.virtualenvs\ai-model-omnia-QHASXsB0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [2]:
corpus = orjsonl.stream('data/open-australian-legal-corpus/corpus.jsonl')

In [3]:
#df = pd.DataFrame(corpus)

In [4]:
#500: 0.27 #10000: 
DOCUMENT_COUNT = 10000  #5000

text = []
for index, document in tqdm(enumerate(corpus)):
    if index < DOCUMENT_COUNT:
        text.append(document['text'])
text_df = pd.DataFrame(data={'text': text})

202260it [00:27, 7404.82it/s] 


In [5]:
text_df.head()

,text
0,Residential Tenancy (Smoke Alarms) Regulations...
1,Explosives Act 2012\n\nAn Act to provide for t...
2,Radiation Protection Act 2005\n\nAn Act for th...
3,FEDERAL COURT OF AUSTRALIA\n\nPEP Community Se...
4,DECISION NO: 67/96\nINDUSTRIAL LAW - INDEPENDE...


In [6]:
#def combine_json(path):
#    
#    folder_path=path
#
#    dataframes = []
#    lungz = []
#
#    for filename in os.listdir(folder_path):
#        if filename.endswith('.json'):
#            file_path = os.path.join(folder_path, filename)
#            with open(file_path, 'r', encoding="utf8") as file:
#                data = json.load(file)
#                if "params" in data:
#                    df = pd.DataFrame(data["params"]["documents"]).apply(pd.Series)
#                else: 
#                    df = pd.DataFrame(data)
#
#                lungh = df.shape[0]
#                lungz.append(lungh)
#                dataframes.append(df)
#
#    if dataframes:
#        combined_df = pd.concat(dataframes, ignore_index=True)
#    else:
#        print('Attenzione, nessun dataframe creato')
#
#    if 'super_category_id' in combined_df.columns:
#        combined_df['super_category_id'] = combined_df['super_category_id'].astype(int)
#
#    return combined_df
#

In [7]:
#folder_path = '.\data'
#combined_df = combine_json(folder_path)
#combined_df

In [8]:
#texts = combined_df["paragraph_body"]
#texts = texts.to_list()
#texts


In [9]:
texts = text_df['text'].to_list()
#texts = [text.lower() for text in texts]
#texts

We will have to train our own tokenizer and build a vocabulary for our corpus. We will be choosing BertWordPieceTokenizer from tokenizers library. 

In [10]:
import tokenizers
 
bwpt = tokenizers.BertWordPieceTokenizer(
        clean_text=True,
        strip_accents=True,
        lowercase=True,
)
 
#filepath = "input file directory"

bwpt.train_from_iterator(
    texts,
    vocab_size=10000, #10000
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000 #1000
)

#doc=5000, vocab=1000: 7.7
#doc=10000, vocab=1000: 15 

In [11]:
bwpt.save('scratch/OmniaBERT')
bwpt.save_model('scratch/', 'OmniaBERT')

['scratch/OmniaBERT-vocab.txt']

In [12]:
# Load the tokenizer
from transformers import BertTokenizer, LineByLineTextDataset

tokenizer_path = 'scratch/OmniaBERT-vocab.txt' 
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

c:\Users\Andrea.Bagante\.virtualenvs\ai-model-omnia-QHASXsB0\Lib\site-packages\transformers\tokenization_utils_base.py:2165: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
c:\Users\Andrea.Bagante\.virtualenvs\ai-model-omnia-QHASXsB0\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
sentence = "federal [MASK] (court) of australia"

encoding = tokenizer.encode(sentence)

print(tokenizer.tokenize(sentence))
print("Token IDs:", encoding)
print("Tokens:", tokenizer.decode(encoding))


['federal', '[MASK]', '(', 'court', ')', 'of', 'australia']
Token IDs: [2, 1591, 4, 12, 495, 13, 278, 891, 3]
Tokens: [CLS] federal [MASK] ( court ) of australia [SEP]


In [14]:
#%%time
#
#'''
#transformers has a predefined class LineByLineTextDataset()
#which reads your text line by line and converts them to tokens
#'''
#
##dataset= LineByLineTextDataset(
##    tokenizer = tokenizer,
##    file_path = '/kaggle/input/bert-bangla/raw_bangla_for_BERT.txt',
##    block_size = 128  # maximum sequence length
##)
#
#dataset = Dataset.from_list([{"text": sentence} for sentence in texts])
#
#print('No. of lines: ', len(dataset)) # No of lines in your datset

In [15]:
from tqdm import tqdm

full_ids = []
attention_masks = []

# For every text...
#for text in tqdm(texts):
#    curr_len = len(text)
#    if curr_len > max_len:
#        max_len = curr_len
#print('ok')
for text in tqdm(texts):
    encoded_dict = tokenizer.encode_plus(
                        text,                      
                        add_special_tokens = True, 
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',     # Return pytorch tensors.

                   )
    full_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

#doc=5000, vocab=1000, max_len=256: 4m
#doc=10000, vocab=1000, max_len=512: 8m

  0%|          | 0/10000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Andrea.Bagante\.virtualenvs\ai-model-omnia-QHASXsB0\Lib\site-packages\transformers\tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 10

In [16]:
full_ids = torch.cat(full_ids, dim=0)
full_ids = torch.tensor(full_ids, dtype=torch.long)
attention_masks = torch.cat(attention_masks, dim=0)
attention_masks = torch.tensor(attention_masks, dtype=torch.float64)

C:\Users\Andrea.Bagante\AppData\Local\Temp\ipykernel_2064\1921582187.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  full_ids = torch.tensor(full_ids, dtype=torch.long)
C:\Users\Andrea.Bagante\AppData\Local\Temp\ipykernel_2064\1921582187.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks = torch.tensor(attention_masks, dtype=torch.float64)


### Creo le maschere

In [17]:
input_ids = full_ids.detach().clone()
rand = torch.rand(input_ids.shape)

# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids > 2) 
for i in range(input_ids.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    input_ids[i, selection] = 4  #[MASK] token == 4 

In [18]:
#input_ids.shape

In [19]:
#input_ids = torch.cat(input_ids, dim=0)
#input_ids = torch.tensor(input_ids, dtype=torch.long)
#input_ids = torch.cat(input_ids, dim=0)

#attention_masks = torch.cat(attention_masks, dim=0)
#attention_masks = torch.tensor(attention_masks, dtype=torch.float64)
#attention_masks = torch.cat(attention_masks, dim=0)

In [20]:
dataset = TensorDataset(input_ids, attention_masks, full_ids)

In [21]:
#torch.save(dataset, 'scratch/dataset')

In [22]:
#dataset = torch.load('scratch/dataset')

First we define the configuration of the BERT model

In [23]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=10000, #10000
    hidden_size=420, #420
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=512 #256
)
#config = BertConfig.from_pretrained("bert-base-cased")
 
model = BertForMaskedLM(config)
#model = BertForMaskedLM.from_pretained('bert-base-cased')

#config = BertConfig(
#vocab_size=tokenizer.vocab_size,
#hidden_size=512,
#num_hidden_layers=6,
#num_attention_heads=8,
#intermediate_size=2048,
#max_position_embeddings=512,
#num_labels=len(unique_labels)
#)

#model = BertForSequenceClassification(config)

print('No of parameters: ', model.num_parameters())


#data_collator = DataCollatorForLanguageModeling(
#    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
#)

No of parameters:  24361972


## Ready to train

# Primo metodo da testare

In [24]:
# Valuto se la GPU è disponibile 
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

2.1.2+cu121
True
cuda:0


In [25]:
model = model.to(device)

In [26]:
dataloader=DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=16)

In [27]:
#dataset[:][0].to('cpu').numpy()

In [28]:
#oeh = next(iter(dataloader))
#oeh[0]
#model.to(device)(oeh[0].to(device), attention_mask=oeh[1].to(device), labels=oeh[2].to(device))

In [29]:
# Training and validation loops

epochs = 4  #4

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()
optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

for epoch_i in range(0, epochs):


    # Training

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(dataloader):
        # Unpack del batch di training dal dataloader, i tensori vengono copiati sulla GPU 
        # il batch contiene due tensori':
        #   [0]: input ids 
        #   [1]: attention masks
        b_input_ids = batch[0]
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_ids = b_input_ids.to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        labels=b_labels)   
        loss = output.loss
        total_train_loss += loss.item()
        loss.backward()

        # Clippa la norma del gradiente a 1.0. Serve per prevenire il problema dell'"exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #da capire questo
        
        optimizer.step()
        scheduler.step()

    # Calcolo della loss media attraverso tutti i batches
    avg_train_loss = total_train_loss / len(dataloader)            
    
    # Controllo quanto tempo ha impiegato questa epoca di allenamento.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...

  Average training loss: 4.39
  Training epoch took: 77.33177328109741

======== Epoch 2 / 4 ========
Training...

  Average training loss: 1.76
  Training epoch took: 77.06180214881897

======== Epoch 3 / 4 ========
Training...

  Average training loss: 1.22
  Training epoch took: 77.80293679237366

======== Epoch 4 / 4 ========
Training...

  Average training loss: 1.11
  Training epoch took: 78.26556134223938

Training complete!
Total training took 310.67945981025696 (h:mm:ss)


In [30]:
save_directory = 'scratch/model_OmniaBERT'

if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
 

model.save_pretrained(save_directory)

In [31]:
from transformers import pipeline

model = BertForMaskedLM.from_pretrained(save_directory)

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)


In [32]:
fill_mask("federal court [MASK] australia")

[{'score': 0.10326690971851349,
  'token': 278,
  'token_str': 'of',
  'sequence': 'federal court of australia'},
 {'score': 0.046051524579524994,
  'token': 0,
  'token_str': '[PAD]',
  'sequence': 'federal court australia'},
 {'score': 0.025952322408556938,
  'token': 273,
  'token_str': 'the',
  'sequence': 'federal court the australia'},
 {'score': 0.02569030225276947,
  'token': 300,
  'token_str': 'and',
  'sequence': 'federal court and australia'},
 {'score': 0.02100345864892006,
  'token': 12,
  'token_str': '(',
  'sequence': 'federal court ( australia'}]

# Secondo metodo

In [ ]:
from transformers import Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir='scratch/',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset # type: ignore
    )


#training_args = TrainingArguments(
#  output_dir='./scratch/',
#  evaluation_strategy="epoch",
#  learning_rate=2e-5,
#  per_device_train_batch_size=16,
#  #per_device_eval_batch_size=16,
#  num_train_epochs=3,
#  weight_decay=0.01,
#)
#
#trainer = Trainer(
#  model=model,
#  args=training_args,
#  train_dataset=final_df["paragraph_text"],
#  #eval_dataset=tokenized_datasets["test"],
#)

In [ ]:
%%time
trainer.train()
trainer.save_model('scratch/')

In [ ]:
from transformers import pipeline

model = BertForMaskedLM.from_pretrained('scratch/')

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)


#model = BertForSequenceClassification.from_pretrained('/scratch/')

#classificazione = pipeline(
#    "text-classification",
#    model=model,
#    tokenizer=tokenizer
#)

In [ ]:
fill_mask("Federal [MASK] of Australia")